In [365]:
import pandas as pd
import numpy as np
import sklearn as sk
import re

In [366]:
data = pd.read_csv('/home/maxim/DS/Phase_0/Phase_0_final_proj/Phase0-Project/tmp_data/data_for_graph')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23368 entries, 0 to 23367
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                23368 non-null  int64  
 1   ID  объявления            23368 non-null  int64  
 2   Количество комнат         22327 non-null  object 
 3   Тип                       23368 non-null  object 
 4   Метро                     22053 non-null  object 
 5   Адрес                     23368 non-null  object 
 6   Площадь, м2               23368 non-null  object 
 7   Дом                       23368 non-null  object 
 8   Парковка                  9951 non-null   object 
 9   Цена                      23368 non-null  float64
 10  Ремонт                    20613 non-null  object 
 11  Площадь комнат, м2        14458 non-null  object 
 12  Балкон                    15390 non-null  object 
 13  Окна                      16755 non-null  object 
 14  Санузе

# Очистка выбросов по цене методом IQR

In [367]:

Q1 = data['Цена'].quantile(0.25)
Q3 = data['Цена'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

upper_bound

np.float64(132000.0)

In [368]:
data = data[(data['Цена'] >= lower_bound) & (data['Цена'] <= upper_bound)]


In [369]:
missing_values_count = data.isnull().sum()
print(missing_values_count)

Unnamed: 0                      0
ID  объявления                  0
Количество комнат            1021
Тип                             0
Метро                        1132
Адрес                           0
Площадь, м2                     0
Дом                             0
Парковка                    12132
Цена                            0
Ремонт                       2371
Площадь комнат, м2           7467
Балкон                       6527
Окна                         5856
Санузел                      2278
Можно с детьми/животными     5480
Дополнительно                 293
Название ЖК                 16031
Серия дома                  18530
Высота потолков, м          10396
Лифт                         4745
Мусоропровод                 8641
Залог                         872
dtype: int64


# Площадь оставить только общую (М)

In [370]:
data.head(5)

,Unnamed: 0,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,...,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Залог
8,8,273973191,3,Квартира,м. Смоленская (9 мин пешком),"Москва, Новинский бульвар, 18С1",120.0/95.0/10.0,"5/10, Сталинский",открытая,130000.0,...,На улицу,Совмещенный (1),Можно с животными,"Мебель на кухне, Ванна, Стиральная машина, Кон...",NaN,NaN,3.0,Пасс (1),Нет,130000.0
11,11,273999490,"3, Изолированная",Квартира,м. Смоленская (2 мин пешком),"Москва, Новинский бульвар, 7",80.0/53.0/11.0,"4/8, Кирпичный",NaN,100000.0,...,На улицу и двор,Совмещенный (1),NaN,"Мебель в комнатах, Мебель на кухне, Душевая ка...",NaN,NaN,NaN,Пасс (1),NaN,110000.0
16,16,274995950,2,Квартира,м. Арбатская (10 мин пешком),"Москва, улица Новый Арбат, 15",30.0/20.0,19/26,NaN,120000.0,...,На улицу,Совмещенный (1),"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,NaN,NaN,NaN,120000.0
48,48,271265359,"2, Оба варианта",Квартира,м. Смоленская (5 мин пешком),"Москва, улица Арбат, 45/24",60.0/43.0/10.0,5/6,наземная,90000.0,...,На улицу и двор,Совмещенный (1),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,90000.0
53,53,273779074,2,Квартира,м. Смоленская (7 мин пешком),"Москва, Большой Николопесковский переулок, 3",60.0/40.0/11.0,"3/6, Кирпичный",NaN,120000.0,...,На улицу,Совмещенный (1),"Можно с детьми, Можно с животными","Мебель в комнатах, Ванна, Стиральная машина, К...",NaN,Индивидуальный проект,3.6,Пасс (1),NaN,120000.0


In [371]:
def roomsplitter(line):
    res = line.split('/')
    return res[0]

print(roomsplitter('198.0/95.0/18.0'))
print(roomsplitter('198.0'))

198.0
198.0


In [372]:
data['Площадь, м2'] = data['Площадь, м2'].apply(roomsplitter).astype('float')

# Привести площадь комнат к формату инт значения (количество комнат) (М)

In [373]:
data['Площадь комнат, м2']

8              45 25 25
11                  NaN
16                  NaN
48                18-25
53                  NaN
              ...      
23363               NaN
23364               NaN
23365               NaN
23366               NaN
23367    26,2+15,5-22,0
Name: Площадь комнат, м2, Length: 20455, dtype: object

In [374]:
data.iloc[30]

Unnamed: 0                                                                203
ID  объявления                                                      273551383
Количество комнат                                                           1
Тип                                                                  Квартира
Метро                                           м. Новодачная (20 мин пешком)
Адрес                       Москва, Дмитровское шоссе, 169К5, ш. Москва-Са...
Площадь, м2                                                              41.2
Дом                                                          13/15, Панельный
Парковка                                                                  NaN
Цена                                                                  36000.0
Ремонт                                                          Косметический
Площадь комнат, м2                                                        NaN
Балкон                                                          

In [375]:
def roomcomposer(line):
    try:
        res = line.replace(' ', ',').replace('-', ',').split(',')
        return res
    except:
        return np.nan

print(roomcomposer('NaN'))
print(roomcomposer('180'))
print(roomcomposer('25-25-20-25'))

['NaN']
['180']
['25', '25', '20', '25']


In [376]:
data['Площадь комнат, м2'].apply(roomcomposer)

8                [45, 25, 25]
11                        NaN
16                        NaN
48                   [18, 25]
53                        NaN
                 ...         
23363                     NaN
23364                     NaN
23365                     NaN
23366                     NaN
23367    [26, 2+15, 5, 22, 0]
Name: Площадь комнат, м2, Length: 20455, dtype: object

# Отфильтровать значения в столбце количество комнат и привести к формату ИНТ

In [377]:
def intchanger(text):
    try:
        res = int(''.join(list(filter(lambda x: x.isdigit() == True, text))))
        return (res)
    except:
        return np.nan

In [378]:
data['Количество комнат'] = data['Количество комнат'].apply(intchanger)

In [379]:
data.head(5)

,Unnamed: 0,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,...,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Залог
8,8,273973191,3.0,Квартира,м. Смоленская (9 мин пешком),"Москва, Новинский бульвар, 18С1",120.0,"5/10, Сталинский",открытая,130000.0,...,На улицу,Совмещенный (1),Можно с животными,"Мебель на кухне, Ванна, Стиральная машина, Кон...",NaN,NaN,3.0,Пасс (1),Нет,130000.0
11,11,273999490,3.0,Квартира,м. Смоленская (2 мин пешком),"Москва, Новинский бульвар, 7",80.0,"4/8, Кирпичный",NaN,100000.0,...,На улицу и двор,Совмещенный (1),NaN,"Мебель в комнатах, Мебель на кухне, Душевая ка...",NaN,NaN,NaN,Пасс (1),NaN,110000.0
16,16,274995950,2.0,Квартира,м. Арбатская (10 мин пешком),"Москва, улица Новый Арбат, 15",30.0,19/26,NaN,120000.0,...,На улицу,Совмещенный (1),"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,NaN,NaN,NaN,120000.0
48,48,271265359,2.0,Квартира,м. Смоленская (5 мин пешком),"Москва, улица Арбат, 45/24",60.0,5/6,наземная,90000.0,...,На улицу и двор,Совмещенный (1),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,90000.0
53,53,273779074,2.0,Квартира,м. Смоленская (7 мин пешком),"Москва, Большой Николопесковский переулок, 3",60.0,"3/6, Кирпичный",NaN,120000.0,...,На улицу,Совмещенный (1),"Можно с детьми, Можно с животными","Мебель в комнатах, Ванна, Стиральная машина, К...",NaN,Индивидуальный проект,3.6,Пасс (1),NaN,120000.0


# Вычислить ремонт по стоимости квартиры   (М)

In [380]:
data[['Цена', 'Ремонт', 'Количество комнат']]

,Цена,Ремонт,Количество комнат
8,130000.0,Евроремонт,3.0
11,100000.0,Евроремонт,3.0
16,120000.0,Евроремонт,2.0
48,90000.0,Евроремонт,2.0
53,120000.0,NaN,2.0
...,...,...,...
23363,42000.0,Евроремонт,NaN
23364,45000.0,Евроремонт,1.0
23365,50000.0,Дизайнерский,2.0
23366,55000.0,Евроремонт,2.0


In [381]:
median_prices = data.groupby(['Количество комнат', 'Ремонт'])['Цена'].median().reset_index()
median_prices.rename(columns={'Цена': 'Медианная_цена'}, inplace=True)

median_prices

,Количество комнат,Ремонт,Медианная_цена
0,1.0,Без ремонта,27000.0
1,1.0,Дизайнерский,55000.0
2,1.0,Евроремонт,40000.0
3,1.0,Косметический,33000.0
4,2.0,Без ремонта,35000.0
5,2.0,Дизайнерский,75000.0
6,2.0,Евроремонт,55000.0
7,2.0,Косметический,42000.0
8,3.0,Без ремонта,46000.0
9,3.0,Дизайнерский,95000.0


In [382]:
missing_values_count = data.isnull().sum()
print(missing_values_count)

Unnamed: 0                      0
ID  объявления                  0
Количество комнат            1021
Тип                             0
Метро                        1132
Адрес                           0
Площадь, м2                     0
Дом                             0
Парковка                    12132
Цена                            0
Ремонт                       2371
Площадь комнат, м2           7467
Балкон                       6527
Окна                         5856
Санузел                      2278
Можно с детьми/животными     5480
Дополнительно                 293
Название ЖК                 16031
Серия дома                  18530
Высота потолков, м          10396
Лифт                         4745
Мусоропровод                 8641
Залог                         872
dtype: int64


In [ ]:
set(median_prices['Ремонт'])

In [383]:
data = data.merge(median_prices, on=['Количество комнат', 'Ремонт'])
(data['Ремонт'])

0           Евроремонт
1           Евроремонт
2           Евроремонт
3           Евроремонт
4         Дизайнерский
             ...      
17095    Косметический
17096       Евроремонт
17097     Дизайнерский
17098       Евроремонт
17099    Косметический
Name: Ремонт, Length: 17100, dtype: object

In [384]:
missing_values_count = data.isnull().sum()
print(missing_values_count)

Unnamed: 0                      0
ID  объявления                  0
Количество комнат               0
Тип                             0
Метро                         961
Адрес                           0
Площадь, м2                     0
Дом                             0
Парковка                     9948
Цена                            0
Ремонт                          0
Площадь комнат, м2           5373
Балкон                       4946
Окна                         4539
Санузел                      1928
Можно с детьми/животными     4461
Дополнительно                 237
Название ЖК                 13839
Серия дома                  16749
Высота потолков, м           8653
Лифт                         3455
Мусоропровод                 5688
Залог                         751
Медианная_цена                  0
dtype: int64


In [ ]:
def fill_repair(row):
    if pd.isna(row['Ремонт']):
        if row['Количество комнат'] == 1:
            repair_mapping = {
            27000.0: 'Без ремонта',
            55000.0: 'Дизайнерский',
            40000.0: 'Евроремонт',
            33000.0: 'Косметический'}

            return repair_mapping.get(round(row['Медианная_цена']), np.nan)
        
        if row['Количество комнат'] == 2:
            repair_mapping = {
            35000.0: 'Без ремонта',
            75000.0: 'Дизайнерский',
            55000.0: 'Евроремонт',
            42000.0: 'Косметический'}

            return repair_mapping.get(round(row['Медианная_цена']), np.nan)
        
        if row['Количество комнат'] == 3:
            repair_mapping = {
            46000.0: 'Без ремонта',
            95000.0: 'Дизайнерский',
            75000.0: 'Евроремонт',
            55000.0: 'Косметический'}

            return repair_mapping.get(round(row['Медианная_цена']), np.nan)
        
        if row['Количество комнат'] == 4:
            repair_mapping = {
            72106.0: 'Без ремонта',
            109995.0: 'Дизайнерский',
            96500.0: 'Евроремонт',
            80000.0: 'Косметический'}

            return repair_mapping.get(round(row['Медианная_цена']), np.nan)
        
        if row['Количество комнат'] == 5:
            repair_mapping = {
            385000.0: 'Без ремонта',
            400000.0: 'Дизайнерский',
            300000.0: 'Евроремонт',
            220000.0: 'Косметический'}

            return repair_mapping.get(round(row['Медианная_цена']), 'Косметический')
        
        if row['Количество комнат'] == 6:
            repair_mapping = {
            700000.0: 'Без ремонта',
            590000.0: 'Дизайнерский',
            550000.0: 'Евроремонт',
            405000.0: 'Косметический'}

            return repair_mapping.get(round(row['Медианная_цена']), 'Косметический')
        
        return 'Косметический'

In [ ]:
data.apply(fill_repair, axis=1)


In [ ]:
missing_values_count = data.isnull().sum()
print(missing_values_count)

# Заполнить колонку "Можно с детьми" 

In [ ]:
data['Можно с детьми/животными'] = data['Можно с детьми/животными'].fillna('Можно с детьми')

In [ ]:
set(data['Можно с детьми/животными'])

In [ ]:
missing_values_count = data.isnull().sum()
print(missing_values_count)

# Удалить серию дома

In [ ]:
data.drop(columns=['Unnamed: 0', 'Серия дома'], inplace=True)

In [ ]:
data.info()

# Заполнить столбец лифт пасажирскими лифтами (М)

In [ ]:
data['Лифт']

In [ ]:
set(data['Лифт'])

In [ ]:
def liftcounter(line):
    try:
        res = re.findall(r'\((\d+)\)', line)
        res = [int(i) for i in res]
        return sum(res)
    except:
        return np.nan

print(liftcounter('Пасс (3), Груз (3)'))

In [ ]:
data['Лифт'] = data['Лифт'].apply(liftcounter)


In [ ]:
data['Лифт']

In [ ]:
Q1 = data['Лифт'].quantile(0.25)
Q3 = data['Лифт'].quantile(0.999)
median = data['Лифт'].agg('median')

print( Q1, Q3, median)

In [ ]:
def liftmeanfiller(value):
    if pd.isna(value):
        return median
    elif value > Q3:
        return median
    return value

print(liftmeanfiller(3.0))

In [ ]:
data['Лифт'] = data['Лифт'].apply(liftmeanfiller)

In [ ]:
set(data['Лифт'])

In [ ]:
missing_values_count = data.isnull().sum()
print(missing_values_count)

#  Заполнить залог данными с цены  (М)

In [ ]:
data.info()

In [ ]:
data['Залог'].fillna(data['Цена'], inplace=True)

# Заполнить парковку

In [ ]:
data['Парковка'] = data['Парковка'].fillna('наземная')

# Удалить мусоропровод

In [ ]:
data.drop(columns=['Мусоропровод'], inplace=True)


# Шаг 5: Заполнение пропусков в остальных колонках

In [ ]:
# Шаг 5: Заполнение пропусков в остальных колонках
basic_columns = ['Балкон', 'Окна', 'Санузел']

for col in basic_columns:
    if col in data.columns:
        # Заполняем самым частым значением
        most_common = data[col].mode()[0] if not data[col].mode().empty else 'Не указано'
        data[col] = data[col].fillna(most_common)
        print(f"Заполнено {col}: {most_common}")

# Удаляем название ЖК

In [ ]:
data.drop(columns=['Название ЖК'], inplace=True)

In [ ]:
data

In [ ]:
# data.to_csv('Data_for_coding')